## (1) SET Environment

In [1]:
import os
os.system('git clone https://github.com/VlOIET/20242R0136COSE47402.git')

0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

In [4]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.4/898.4 kB 17.4 MB/s eta 0:00:00


In [5]:
pip install transformers huggingface-hub

In [6]:
from huggingface_hub import login
from google.colab import userdata
HF_Token = userdata.get('HF_AT')

login(token=HF_Token)

In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="XM87RY8f7oIaykIwsDoU")
project = rf.workspace("food-recipe-ingredient-images-0gnku").project("food-ingredients-dataset")
version = project.version(4)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to FOOD-INGREDIENTS-dataset-4 in yolov8:: 100%|██████████| 19480/19480 [00:03<00:00, 6034.53it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## (2) Execute Model

In [8]:
#Environment
DEVICE = "cuda"
SAVE_FOLDER = "/content/drive/MyDrive/DeepLearning/Final_Project/Predict_Result"
PREDICT_PATH = "/content/drive/MyDrive/DeepLearning/Final_Project/predict_result.txt"

YOLO_MODEL_NAME = "/content/20242R0136COSE47402/FinalProject/Output models/YOLO/1205best.pt"
llama_MODEL_PATH = "/content/20242R0136COSE47402/FinalProject/Output models/llama/1205"

In [9]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from transformers import AutoTokenizer, AutoModelForCausalLM

In [10]:
llm_model = AutoModelForCausalLM.from_pretrained(llama_MODEL_PATH).to(DEVICE)
llm_tokenizer = AutoTokenizer.from_pretrained(llama_MODEL_PATH)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [11]:
folder_path = "/content/drive/MyDrive/DeepLearning/Final_Project/Test_Image"

file_list = os.listdir(folder_path)
file_num = 0

for file_name in file_list:
  file_path = os.path.join(folder_path, file_name)
  DATA_PATH = file_path
  SAVE_PATH = os.path.join(SAVE_FOLDER, f"{file_num}.png")
  file_num += 1

  model = YOLO(YOLO_MODEL_NAME)
  results = model.predict(DATA_PATH)
  res_plot = results[0].plot()
  cv2.imwrite(SAVE_PATH, res_plot)

  ingredient_list =[]

  for result in results:
    labels = [model.names[int(cls)] for cls in result.boxes.cls]
    ingredient_list.extend(labels)

  input_ingredients = ", ".join(ingredient_list)

  input_text = (
      f"You are an expert chef. Based on the given ingredients, suggest a recipe title and step-by-step cooking directions.\n\n"
      f"Ingredients:\n{input_ingredients}\n\n"
      f"Directions:"
  )

  input_ids = llm_tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)
  outputs = llm_model.generate(
          input_ids,
          max_length=input_ids.shape[1] + 300,  # Relative max length
          max_new_tokens=300,
          num_return_sequences=1,
          do_sample=True,
          temperature=0.1,
          top_p=0.9,
          top_k=50,
          repetition_penalty=1.2,
          no_repeat_ngram_size=2,
          eos_token_id=llm_tokenizer.eos_token_id
  )
  result = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

  with open(PREDICT_PATH, "a") as file:
      file.write(result + "\n")


image 1/1 /content/drive/MyDrive/DeepLearning/Final_Project/Test_Image/-19_jpg.rf.65eb10667b4c6c4a826eefca7fdc7176.jpg: 640x640 1 Capsicum, 10.1ms
Speed: 12.7ms preprocess, 10.1ms inference, 743.6ms postprocess per image at shape (1, 3, 640, 640)


Both `max_new_tokens` (=300) and `max_length`(=347) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



image 1/1 /content/drive/MyDrive/DeepLearning/Final_Project/Test_Image/034_PNG_jpg.rf.c790a87bef605090c91ca5fe5200307f.jpg: 640x640 1 Chili Powder, 1 Onion, 1 mayonnaise, 10.1ms
Speed: 1.7ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


Both `max_new_tokens` (=300) and `max_length`(=355) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



image 1/1 /content/drive/MyDrive/DeepLearning/Final_Project/Test_Image/059_PNG_jpg.rf.dce7459c30dde9a8037ade8d4a613e21.jpg: 640x640 1 Cheese, 1 Onion Leaves, 12.8ms
Speed: 1.9ms preprocess, 12.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


...

Both `max_new_tokens` (=300) and `max_length`(=349) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



image 1/1 /content/drive/MyDrive/DeepLearning/Final_Project/Test_Image/search.pstatic (1).jpg: 640x480 (no detections), 60.1ms
Speed: 2.1ms preprocess, 60.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)


Both `max_new_tokens` (=300) and `max_length`(=344) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



image 1/1 /content/drive/MyDrive/DeepLearning/Final_Project/Test_Image/search.pstatic (2).jpg: 480x640 (no detections), 54.4ms
Speed: 2.0ms preprocess, 54.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)


Both `max_new_tokens` (=300) and `max_length`(=344) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
